# Loading Data

This tutorial shows how to load and understand the GEOM data. **Caution: We will only be updating the RDKit files as new data gets added to GEOM, and we will not be updating the messagepack files. Make sure you are using the RDKit files if you want the most up-to-date data.**


## Un-tarring the files
To extract the tarred files, run
``` bash
tar -xzvf <name_of_file>
```
on the command line for all four of the file names. This will extract the files so that they can be loaded in this tutorial.

## Crude data
We'll start with an example of original CREST information for drugs, contained in the file `drugs_crude.msgpack`.

First we load `msgpack`:

In [1]:
import msgpack

Next we call `msgpack.Unpacker` to load the file:

In [3]:
import os

# change to where your data is located
direc = "/home/saxelrod/rgb_nfs/GEOM_NON_TAR"

drugs_file = os.path.join(direc, "drugs_crude.msgpack")
unpacker = msgpack.Unpacker(open(drugs_file, "rb"))


The file is broken up into 292 dictionaries, each containing information for approximately 1000 molecules. `Unpacker` loads these dictionaries one-at-a-time, so that you don't need to keep a big dictionary for 292,000 molecules in memory at one time.

To see how this works, let's say we just want to look at 1000 molecules. Then we can call `next(iter(unpacker))` to load the first dictionary:

In [4]:
drugs_1k = next(iter(unpacker))

If we want to load all the dictionaries, we can initialize a dictionary, loop over the unpacker, and update the dictionary as we go through the loop (note that this will take several minutes to load and should be commented out to quickly go through the tutorial):

In [5]:
# overall_drug_dic = {}
# for drug_dic in unpacker:
#     overall_drug_dic.update(drug_dic)

Back to the first dictionary, it indeed has approximately 1000 entries:

In [6]:
print(len(drugs_1k))

982


The structure of the dictionary is as follows:

\begin{align}
\mathrm{SMILES} \to     \begin{cases}
        \text{conformers} \to \quad\quad \begin{cases} \text{conf. 0} \\ \text{conf. 1} \to \begin{cases} \text{xyz} \\ \text{statistical weight } \\ \text{...} \end{cases}\\ \text{...} \end{cases} \\
      \text{ensemble entropy} \\ \text{average energy} \\ \text{SARS-CoV 3CL active} \\ \text{...} \end{cases}
    %   \text{Conformer list:  \quad\quad\  } \begin{cases} \text{Conf. 0 } \\ \text{Conf. 1} \to \begin{cases} \text{xyz} \\ \text{statistical weight} \\ \text{...} \end{cases} \\ \text{...} \end{cases}
    % \end{cases}  
\end{align}


Each dictionary key is a SMILES string, and each value is a sub-dictionary. Apart from the `conformers` key, every key in the sub-dictionary corresponds to a property of the molecule as a whole, rather than one individual conformer. This includes the energy averaged over all conformers, whether the molecule binds a specific protein, and so on. The `conformer` key, on the other hand, contains information about the set of individual conformers, including their coordinates (`xyz`), individual energy, and so on.

For an example let's look at the first dictionary entry, but leave out `conformers`:

In [7]:
sample_smiles = list(drugs_1k.keys())[10]
sample_sub_dic = drugs_1k[sample_smiles]
print(sample_smiles)
{key: val for key, val in sample_sub_dic.items() if key != 'conformers'}

O=C(/C=C/c1ccco1)Nc1ccc(Cl)c(S(=O)(=O)N2CCOCC2)c1


{'totalconfs': 127,
 'temperature': 298.15,
 'uniqueconfs': 84,
 'lowestenergy': -79.0497,
 'poplowestpct': 45.123,
 'ensembleenergy': 0.869,
 'ensembleentropy': 5.871,
 'ensemblefreeenergy': -1.75,
 'sars_cov_one_pl_protease_active': 0,
 'sars_cov_one_cl_protease_active': 0,
 'charge': 0,
 'datasets': ['plpro', 'aid1706']}

We see that:
- The SMILES `O=C(/C=C/c1ccco1)Nc1ccc(Cl)c(S(=O)(=O)N2CCOCC2)c1` has 127 total conformers (`totalconfs`), including rotamers, but only 84 unique conformers, i.e. excluding conformers (`uniqueconfs`). 
- The energy of the lowest-energy conformer is given by `lowestenergy` as -79.0497 Hartree. 
- The highest-population conformer (`poplowestpct`) has a 45.123% occupation probability. 
- The average energy of all the conformers, taking into account their statistical weights, is given by `ensembleenergy` as 0.869 kcal/mol. 
- The conformational entropy (`ensembleentropy`) is 5.871 cal / mol K, while the conformational free energy ($-T S$, where $T$ is 298.15 K, as given by `temperature` and $S$ is the entropy) is -1.75 kcal.
- The species is inactive for the binding of SARS-CoV(1) PL protease and SARS-CoV(1) Cl protease (if it were active for the former, then the value of `sars_cov_one_pl_protease_active` would be 1).
- The charge is 0
- This molecule can be found in the datasets `plpro` and `aid1706`.

The references for the different dataset names can be found in the **References** section of this tutorial.

Next let's take take a look at some of the conformers:

In [8]:
len(sample_sub_dic["conformers"])

84

There are 84 conformers, as expected from the fact that `uniqueconfs` is 84, above. Each conformer value is a dictionary. Let's look at the first one:


In [9]:
sample_sub_dic["conformers"][0]

{'xyz': [[8.0, -3.3319635422, -2.0442902558, 0.6909756987],
  [6.0, -3.2168333749, -0.9206740514, 0.2452514931],
  [7.0, -2.0069577355, -0.3545300637, -0.0988018889],
  [6.0, -0.7379808342, -0.9107855662, -0.032150557],
  [6.0, 0.3460883674, -0.1248891969, -0.4278331255],
  [6.0, 1.6279270623, -0.629676906, -0.3872858108],
  [6.0, 1.8618600425, -1.9237381238, 0.0467393099],
  [6.0, 0.7913564976, -2.7144979703, 0.4455286077],
  [6.0, -0.4957464401, -2.2147163131, 0.4076787986],
  [1.0, -1.3270616821, -2.8278196518, 0.7150750074],
  [1.0, 0.9728504968, -3.7220742003, 0.7830047425],
  [17.0, 3.4634297054, -2.524649522, 0.0822810097],
  [16.0, 2.9835266829, 0.4198969323, -0.9208824315],
  [8.0, 3.7236332953, -0.2310199926, -1.9569565476],
  [8.0, 2.4216194889, 1.7132399647, -1.2025823471],
  [7.0, 3.9128944837, 0.4383716437, 0.4709954112],
  [6.0, 5.3520760488, 0.389959575, 0.2952279212],
  [1.0, 5.5561982774, -0.2690412476, -0.5478406962],
  [6.0, 5.9053926436, 1.7912620402, 0.0307937815]

Let's go through the keys one-by-one:
- `xyz`: the coordinates of the atoms in the molecule. It is a list of the form [[atom_0_atomic_number, atom_0_x, atom_0_y, atom_0_z], [atom_1_atomic_number, atom_1_x, atom_1_y, atom_1_z], ...]
- `geom_id`: a unique number identifying every geometry
- `set`: the index of the MD set that this geometry came from during the CRET run.
- `degeneracy`: how many degenerate rotamers there are for this conformer.
- `totalenergy`: the absolute energy of this conformer, in Hartree.
- `relativeenergy`: the energy of this conformer, in kcal/mol, relative to the lowest-energy conformer. Here it is 0 because it is the lowest energy conformer.
- `boltzmannweight`: statistical weight of this conformer. It is equal to $d \ e^{-E/k_{\mathrm{B}} T}   /  ( \sum_i d_i e^{-E_i / k_{\mathrm{B}} T})$, where $d$ is the degeneracy, $E$ is the energy, and the sum in the denominator is overa all conformers.
- `conformerwights`: the weights associated with each rotamer of this conformer. Weights should be equal for different rotamers, as they are in principle equivalent, but in practice small differences are produced due to CREST's finite accuracy.


And that's almost all there is to the crude data! 

Before we move on, just remember two things:
1. Not every key is in every dictionary. For example, not every molecule is labelled with whether or not it binds SARS-CoV 3CL. If you want to train a model on molecules with one of those properties, make sure to first filter out species that do not have them.

2. Even though the conformers are ordered by energy in this example, **this is not always the case! Conformers can be ordered in a way that is not related to their energy or statistical weights!** So be careful of this. If you want to use the $n$ lowest conformers, make sure to first sort them by statistical weights and then take the first $n$.

## Featurized Data

Now we'll take a look at some of the featurized data. For an example we'll load the first chunk of 1000 featurized molecules from QM9. 

Note that loading the drug features is very slow, as there is a significant amount of data to be loaded. If it's taking a long time to load, don't worry! That's perfectly normal. 

However, you may not be able to store features for all the molecules in memory at the same time. If you want to perform analysis on the features (say, by converting them to feature vectors), we recommend that you load the dictionaries one-by-one, perform the analysis, and save the results to disk. That way you will only have to hold features for 1000 molecules in memory at a time.

In [13]:
qm9_features_file = os.path.join(direc, "qm9_featurized.msgpack")
qm9_unpacker = msgpack.Unpacker(open(qm9_features_file, "rb"))
qm9_feat_1k = next(iter(qm9_unpacker))

The structure of the featurized data dictionaries is as follows:

\begin{align}
\mathrm{SMILES} \to     \begin{cases}
\mathrm{conf_0 } \\ \mathrm{conf_1 } \to \begin{cases} \text{atom features} \\ \text{bond features } \\ \text{5 A neighbor list} \\ \text{xyz2mol smiles } \\ \text{canonical xyz2mol smiles }  \end{cases}\\  \text{...}  \end{cases} 
\end{align}


So each SMILES key leads to a value that is a list of dictionaries, one for each conformer. That dictionary contains atom features, bond features, the 5 Angstrom neighbor list, the SMILES string generated by `xyz2mol`, and the canonical form of that SMILES string. Because reactivity may occur during CREST, even the canonical form of the `xyz2mol` SMILES may not be the same as the original SMILES key.

Let's take a look at what's in the sub-dictionary of the first SMILES string:

In [14]:
qm9_smiles = list(qm9_feat_1k.keys())[10]
qm9_sub_dic = qm9_feat_1k[qm9_smiles]
print(qm9_smiles)
qm9_feat_1k[qm9_smiles][0]

CN1C[C@H]2NC[C@]21C#N


{'bonds': [{'indices': [0, 1],
   'type': 'single',
   'stereo': 'none',
   'conjugated': False,
   'in_ring': False,
   'ring_size': -1},
  {'indices': [0, 15],
   'type': 'single',
   'stereo': 'none',
   'conjugated': False,
   'in_ring': False,
   'ring_size': -1},
  {'indices': [0, 16],
   'type': 'single',
   'stereo': 'none',
   'conjugated': False,
   'in_ring': False,
   'ring_size': -1},
  {'indices': [0, 17],
   'type': 'single',
   'stereo': 'none',
   'conjugated': False,
   'in_ring': False,
   'ring_size': -1},
  {'indices': [1, 2],
   'type': 'single',
   'stereo': 'none',
   'conjugated': False,
   'in_ring': True,
   'ring_size': 4},
  {'indices': [1, 10],
   'type': 'single',
   'stereo': 'none',
   'conjugated': False,
   'in_ring': True,
   'ring_size': 4},
  {'indices': [2, 3],
   'type': 'single',
   'stereo': 'none',
   'conjugated': False,
   'in_ring': False,
   'ring_size': -1},
  {'indices': [2, 4],
   'type': 'single',
   'stereo': 'none',
   'conjugated': 

The `bonds` key maps to a list of dictionaries, each of which gives features for a different bond:
- `indices`:tells you the ordered indices of atoms involved in the bond
- `type`: the bond type (single, double, triple, or aromatic)
- `stereo`:the stereochemistry of the bond ("stereonone", "stereoany", "stereoz","stereoe", "stereocis", or "stereotrans")
- `conjugated` whether or not the bond is part of a conjugated system
- `in_ring`: whether the bond is part of a ring
- `ring_size`: size of the ring the bond is part of. If it is not part of a ring, then this value is -1.

The `atoms` key maps to a list of dictionaries, each of which gives features for a different atom:
- `atom_type`: atomic number of the atom
- `num_bonds`: number of bonds the atom is involved in
- `formal_charge`: charge on the atom
- `chirality`: chirality of the atom ("chi_unspecified", "chi_tetrahedral_cw", "chi_tetrahedral_ccw", or "chi_other")
- `num_bonded_h`: number of hydrogens bonded to this atom
- `hybridization`: hybridization ("s", "sp", "sp2", "sp3", "sp3d", or  "sp3d2")
- `aromatic`: whether the atom is part of an aromatic system
- `mass`: mass of the most common isotope of the atom, in amu.

The `nbrs_5A` key gives an undirected neighbour list with a 5 Angstrom cutoff. Note that this includes all two atoms within 5 Angstrom of each other, even if they are not bonded.


**Remember**: while most molecules in the `crude` files are also in the `featurized` files, there are some missing because of graph conversion errors. So if a molecule seems like it's missing, then it may be! But luckily that's a very small minority.

And there you have it, you're ready to use GEOM!

# References


Here we provide the references for the different possible values of the `dataset` key in the dictionaries:

- `ellinger`:
    - Bernhard Ellinger, Denisa Bojkova, Andrea Zaliani, Jindrich Cinatl, Carsten Claussen, Sandra Westhaus, Jeanette Reinshagen, Maria Kuzikov, Markus Wolf, Gerd Geisslinger, et al. Identification of inhibitors of SARS-CoV-2 in-vitro cellular toxicity in human (Caco-2) cells using a large scale drug repurposing collection. 2020.

- `amu_sars_cov_2`: 
    - Franck Touret,Magali Gilles, Karine Barral, Antoine Nougairède, Etienne Decroly, Xavierde Lamballerie, and Bruno Coutard. In vitro screening of a FDA approved chemical library reveals potential inhibitors of SARS-CoV-2 replication. BioRxiv, 2020.
    
- `mpro_xchem`: 
    - Main protease structure and XChem fragment screen. https://www.diamond.ac.uk/covid-19/for-scientists/Main-protease-structure-and-XChem.html.
   
    
- `aid1706`: 
    - Valerie Tokars and Andrew Mesecar. QFRET-based primary biochemical high throughput screening assayto identify inhibitors of the SARS coronavirus 3C-like Protease (3CLPro). https://pubchem.ncbi.nlm.nih.gov/bioassay/1706
    - https://github.com/yangkevin2/coronavirus_data/blob/master/data/AID1706_binarized_sars.csv. Accessed: 2020-03-28
    
- `plpro`: 
    - Daniel Engel.  qHTS of yeast-based assay for SARS-CoV PLP. https://pubchem.ncbi.nlm.nih.gov/bioassay/485353
    - Daniel Engel. qHTS of yeast-based assay for SARS-CoV PLP: Hit validation. https://pubchem.ncbi.nlm.nih.gov/bioassay/652038.
- `broad_repurposing_library`: 
    - Drug Repurposing Hub.https://www.broadinstitute.org/drug-repurposing-hub.
- `ecoli`:
    - Mattia Zampieri, Michael Zimmermann, Manfred Claassen, and Uwe Sauer. Nontargeted metabolomicsreveals the multilevel response to antibiotic perturbations.Cell reports, 19(6):1214–1228, 2017. https://www.sciencedirect.com/science/article/pii/S2211124717304618
    - Jonathan M Stokes, Kevin Yang, Kyle Swanson, Wengong Jin, Andres Cubillos-Ruiz, Nina M Donghia,Craig R MacNair, Shawn French, Lindsey A Carfrae, Zohar Bloom-Ackerman, et al.  A deep learningapproach to antibiotic discovery. Cell, 180(4):688–702, 2020. https://www.sciencedirect.com/science/article/pii/S0092867420301021
- `pseudomonas`:
    - https://www.aicures.mit.edu/data. Accessed: 2020-05-22. (Data available on request)
    


